In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
#dbutils.secrets.listScopes()

In [0]:
#dbutils.secrets.list(scope="databricksscope")

In [0]:
spark.conf.set("fs.azure.account.key.optumsadllssa.dfs.core.windows.net",dbutils.secrets.get(scope="databricksscope",key="adlskey"))

In [0]:
#display(dbutils.fs.ls("abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net"))

In [0]:
data = spark.read.csv('abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/group.csv',header=True,inferSchema=True)

In [0]:
data.show(5,False)

+-------+---------------+--------+------+----------------------------------------+--------+------+
|country|premium_written|zip_code|grp_id|grp_name                                |grp_type|city  |
+-------+---------------+--------+------+----------------------------------------+--------+------+
|India  |72000          |482018  |GRP101|Life Insurance Corporation of India     |Govt.   |Mumbai|
|India  |45000          |482049  |GRP102|HDFC Standard Life Insurance Co. Ltd.   |Private |Mumbai|
|India  |64000          |482030  |GRP103|Max Life Insurance Co. Ltd.             |Private |Delhi |
|India  |59000          |482028  |GRP104|ICICI Prudential Life Insurance Co. Ltd.|Private |Mumbai|
|India  |37000          |482014  |GRP105|Kotak Mahindra Life Insurance Co. Ltd.  |Private |Mumbai|
+-------+---------------+--------+------+----------------------------------------+--------+------+
only showing top 5 rows



In [0]:
data.count()

58

In [0]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+-------+---------------+--------+------+--------+--------+----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|
+-------+---------------+--------+------+--------+--------+----+
|      0|              0|       0|     0|       0|       0|   0|
+-------+---------------+--------+------+--------+--------+----+



In [0]:
data.groupby(['country','premium_written','zip_code','grp_id','grp_name','grp_type','city']).count().where("count>1").show()

+-------+---------------+--------+------+--------+--------+----+-----+
|country|premium_written|zip_code|grp_id|grp_name|grp_type|city|count|
+-------+---------------+--------+------+--------+--------+----+-----+
+-------+---------------+--------+------+--------+--------+----+-----+



In [0]:
data = data.dropDuplicates()

In [0]:
data.show()

+-------+---------------+--------+------+--------------------+--------+---------+
|country|premium_written|zip_code|grp_id|            grp_name|grp_type|     city|
+-------+---------------+--------+------+--------------------+--------+---------+
|  India|          70000|  482006|GRP107|TATA AIG Life Ins...| Private|   Mumbai|
|  India|          47000|  482046|GRP119|Canara HSBC Orien...| Private| Gurugram|
|  India|          79000|  482036|GRP115|Shriram Life Insu...| Private|Hyderabad|
|  India|          57000|  482011|GRP111|PNB MetLife India...| Private|   Mumbai|
|  India|          50000|  482017|GRP120|Aegon Life Insura...| Private|   Mumbai|
|  India|          33000|  482043|GRP114|Sahara India Life...| Private|  Lucknow|
|  India|          64000|  482030|GRP103|Max Life Insuranc...| Private|    Delhi|
|  India|          89000|  482011|GRP106|Aditya Birla Sun ...| Private|   Mumbai|
|  India|          45000|  482049|GRP102|HDFC Standard Lif...| Private|   Mumbai|
|  India|       

In [0]:

output_container_path = "abfss://optumstagingdata@optumsadllssa.dfs.core.windows.net"
output_blob_folder = "optumstagingdata"
data.coalesce(1).write.format("csv").mode("overwrite").option("header","true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path,"%s/groupdatastaging.csv"%output_container_path)

True